In [3]:
from data import load_dataset, load_dataset_as_complex, graph_to_complex
from time import time

In [4]:
st = time()
dataset = load_dataset()
print('Dataset loaded in', time() - st, 's')

st = time()
complex_dataset = load_dataset_as_complex()
print('Dataset loaded in', time() - st, 's')

Dataset loaded in 8.033021926879883 s
Dataset loaded in 7.849362850189209 s


In [5]:
for data, exp in dataset:
    print(data)
    break

Data(x=[24, 14], edge_index=[2, 52], edge_attr=[52, 5], y=[1])


In [6]:
for data, exp, mapping in complex_dataset:
    print(mapping)
    break

{1: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51}, 2: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51}, 3: {0: [31, 50, 37, 35, 33], 1: [31, 50, 37, 35, 33], 2: [31, 50, 37, 35, 33], 3: [31, 50, 37, 35, 33], 4: [31, 50, 37, 35, 33], 5: [1, 46, 9, 7, 5, 3], 6: [1, 46, 9, 7, 5, 3], 7: [1, 46, 9, 7, 5, 

In [7]:
st = time()
for data, exp in dataset:
    graph_to_complex(data)
et = time()
print('Graph to complex in', et - st, 's')
print('Graph to complex in', (et - st) / len(dataset), 's per graph')

Graph to complex in 1.97062087059021 s
Graph to complex in 0.0001642184058825175 s per graph


In [8]:
from data import get_data_loaders


train_loader, test_loader = get_data_loaders(dataset, batch_size=64)
complex_train_loader, complex_test_loader = get_data_loaders(complex_dataset, batch_size=64)

In [9]:
for data in train_loader:
    print(data)
    break

DataBatch(x=[1301, 14], edge_index=[2, 2740], edge_attr=[2740, 5], y=[64], exp_key=[64], batch=[1301], ptr=[65])


In [10]:
for data in complex_train_loader:
    print(data)
    break

DataBatch(y=[64], edge_index=[2, 10078], x=[4297, 14], node_type=[4297], edge_type=[10078], batch=[4297], ptr=[65])


In [11]:
import os

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GINConv, GATConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, in_dim=None, hidden_dim=None, out_dim=None):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = torch.nn.Linear(hidden_dim, out_dim)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = global_mean_pool(x, batch)
        x = self.lin(x)
        # sigmoid
        x = F.sigmoid(x)
        x = x.squeeze(1)
        return x

In [12]:
model = GCN(in_dim=14, hidden_dim=64, out_dim=1)

# train
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
st = time()
for epoch in range(50):
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = F.binary_cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
    print(epoch, loss, end='\r')
et = time()
print('Training in', et - st, 's')

Training in 35.568952798843384 sossEntropyBackward0>)


In [13]:
complex_model = GCN(in_dim=14, hidden_dim=64, out_dim=1)

# train
optimizer = torch.optim.Adam(complex_model.parameters(), lr=0.01, weight_decay=5e-4)
complex_model.train()
st = time()
for epoch in range(50):
    for data in complex_train_loader:
        optimizer.zero_grad()
        out = complex_model(data.x, data.edge_index, data.batch)
        loss = F.binary_cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
    print(epoch, loss, end='\r')
et = time()
print('Training in', et - st, 's')

Training in 99.3596568107605 sCrossEntropyBackward0>)
